In [1]:
import random
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras import Model
from tensorflow.keras import layers
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from ISR.models import Discriminator
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
DATADIR = "galaxy_zoo"
CATEGORIES = ["individuals", "merged"]

In [3]:
FOLDER = []
for i in [2, 3, 4]:
    for c in CATEGORIES:
        for t in ["valid", "train"]:
            FOLDER.append(c+"_"+str(i)+"blend_"+t)

training_data = []

for f in FOLDER:
    path = os.path.join(DATADIR, f)
    class_num = 0
    if CATEGORIES[1] in f:
        class_num = 1

    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(
                path, img), cv2.IMREAD_UNCHANGED)
            new_array = cv2.resize(img_array, (80, 80))

            training_data.append([new_array, class_num])
        except Exception as e:
            pass

random.shuffle(training_data)

In [4]:
batch_size = 12
IMG_SIZE = 80

discr = Discriminator(patch_size=IMG_SIZE, kernel_size=3)
# m = layers.Flatten()(discr.model.output)
# m = layers.Dense(1, activation="sigmoid")(m)
# model = Model(discr.model.input, m)
discr_out_shape = list(discr.model.outputs[0].shape)[1:4]

In [5]:
X = []
y = []
z = None
for features, label in training_data:
    X.append(features)

    if label == 1:
        # z=[0]*400
        z = np.zeros(discr_out_shape)
    else:
        # z=[1]*400
        z = np.ones(discr_out_shape)

    y.append(z)




X = np.array(X)
y = np.array(y)

In [6]:
X = np.array(X).reshape(-1, 80, 80, 3)
y = np.array(y).reshape(-1, 20, 20, 1)

In [7]:
X.shape

(120000, 80, 80, 3)

In [8]:
y.shape

(120000, 20, 20, 1)

In [9]:
discr_out_shape

[20, 20, 1]

In [10]:
discr.model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

filepath = "model/21/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, save_weights_only=True, mode='max')
tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()))

In [ ]:
discr.model.fit(X, y,
                batch_size=batch_size,
                epochs=10000,
                validation_split=0.3,
                callbacks=[tensorboard, checkpoint])

Epoch 1/10000
6999/7000 [============================>.] - ETA: 0s - loss: 0.0641 - accuracy: 0.9794
Epoch 00001: val_accuracy improved from 0.96425 to 0.98163, saving model to model/21/weights-improvement-01-0.98.hdf5
7000/7000 [==============================] - 269s 38ms/step - loss: 0.0641 - accuracy: 0.9794 - val_loss: 0.0599 - val_accuracy: 0.9816
Epoch 2/10000
6999/7000 [============================>.] - ETA: 0s - loss: 0.0521 - accuracy: 0.9836
Epoch 00002: val_accuracy improved from 0.98163 to 0.98306, saving model to model/21/weights-improvement-02-0.98.hdf5
7000/7000 [==============================] - 271s 39ms/step - loss: 0.0521 - accuracy: 0.9836 - val_loss: 0.0529 - val_accuracy: 0.9831
Epoch 3/10000
7000/7000 [==============================] - ETA: 0s - loss: 0.0440 - accuracy: 0.9862
Epoch 00003: val_accuracy improved from 0.98306 to 0.98352, saving model to model/21/weights-improvement-03-0.98.hdf5
7000/7000 [==============================] - 282s 40ms/step - loss: 0.0